In [17]:
using Pkg

In [18]:
using Polyhedra

In [19]:
using MeshCat

In [20]:
using JuMP

In [21]:
using GLPK

In [26]:
function ObjectiveIO()
    println("Enter the coefficients to a linear objective function. Keep in mind, a function with only 3 variables can be accepted.")
    println("For example, if you had the objective function: ")
    println(".75x+.5y+.1z = 1")
    println("You would input: ")
    println(".75 .5 .1")
    objective = Matrix{Float64}(undef,0,0)
    input = readline(stdin)
    r = hcat(parse.(Float64, split(input))...)
    objective = size(objective,1)>0 ? vcat(objective,r) : r

    objective = vec(objective)
    display(objective)

    println("Enter what the objective function equals")
    println("For example, if you had the objective function: ")
    println(".75x+.5y+.1z = 1")
    println("You would input: ")
    println("1")
    equals = readline(stdin)
    equals = parse.(Float64, equals)
    println(equals)
    return objective, equals
end

ObjectiveIO (generic function with 1 method)

In [31]:
function PolyhedraIOHRep()
    println("Enter the values for the coefficient matrix A of the system of linear inequalities. Keep in mind, an inequality with only 3 variables can be accepted.")
    println("For example if your system looks like this: ")
    println("-x ≤ 0\n-y ≤ 0\n-z ≤ 0\n35x+.5y+.5z ≤ .5\n60x+300y+10 ≤ 15\n30x+20y+10z ≤ 4")
    println("You would input: ")
    println("-1 0 0\n0 -1 0\n0 0 -1\n35 .5 .5\n 60 300 10\n30 20 10")
    println("For the coefficient matrix. When you are done entering elements, press \"ENTER\" on the blank readline")
    println("KEEP IN MIND: The entries must be in the form of Ax ≤ b")
    A = Matrix{Float64}(undef,0,0)

    while true
        s = readline(stdin)
        if length(s) <1
            break
        end

        r = hcat(parse.(Float64, split(s))...)
        A = size(A,1)>0 ? vcat(A,r) : r
    end
    display(A)


    println("Enter the values for vector b of the system of linear inequalities")
    println("For example if your system looks like this: ")
    println("-x ≤ 0\n-y ≤ 0\n-z ≤ 0\n35x+.5y+.5z ≤ .5\n60x+300y+10 ≤ 15\n30x+20y+10z ≤ 4")
    println("You would input: ")
    println("0 0 0 .5 15 4")
    println("For b")
    println("KEEP IN MIND: The entries must be in the form of Ax ≤ b")
    b = Matrix{Float64}(undef,0,0)

    s = readline(stdin)


    r = hcat(parse.(Float64, split(s))...)
    b = size(b,1)>0 ? vcat(b,r) : r

    b = vec(b)
    display(b)
   
    return A,b
end

PolyhedraIOHRep (generic function with 1 method)

In [8]:
function VertexIO()
    lib = DefaultLibrary{Float64}(GLPK.Optimizer)
    println("Enter the verticies for your polyhedron. Keep in mind, a point with only 3 variables can be accepted.")
    println("For example, if you had a polyhedron consisting of the points")
    println("(1,0,0)\n(0,1,0)\n(0,0,1)\n(-1,0,0)\n(0,-1,0)\n(0,0,-1)")
    println("You would input: ")
    println("1 0 0\n0 1 0\n 0 0 1\n -1 0 0\n 0 -1 0\n 0 0 -1")
    println("When you are done entering verticies, press \"ENTER\" on the blank readline")
    
    verticies = Matrix{Float64}(undef,0,0)

    while true
        s = readline(stdin)
        if length(s) < 1
            break
        end
            r = hcat(parse.(Float64, split(s))...)
            verticies = size(verticies,1)>0 ? vcat(verticies,r) : r
    end
    vrepPoly = polyhedron(vrep(verticies),lib)
    return vrepPoly
end

VertexIO (generic function with 1 method)

In [9]:
function MakePolyhedra(A, b)
    hrepPoly = HalfSpace(A[1,:],b[1])

    for i in 2:length(b)
        hrepPoly = hrepPoly ∩ HalfSpace(A[i,:],b[i])
    end
    display(hrepPoly)
    
    return hrepPoly
end

MakePolyhedra (generic function with 1 method)

In [10]:
function convertToVRep(hrepPoly)
    println("Your polyhedron in V-representation looks like:")
    polyhedraH = polyhedron(hrepPoly)
    VR = vrep(polyhedraH)
    display(VR)
    return VR
end

convertToVRep (generic function with 1 method)

In [11]:
function ConvertToHRep(vrepPoly)
    println("Your polyhedron in H-representation looks like:")
    VR = vrep(vrepPoly)
    polyhedraV = polyhedron(VR)
    HR = hrep(polyhedraV)
    display(HR)
    return HR
end

ConvertToHRep (generic function with 1 method)

In [12]:
function DisplayPolyhedra(hrepPoly)
    
    polyhedra = polyhedron(hrepPoly)
    polyhedraMesh = Polyhedra.Mesh(polyhedra)
    vis = Visualizer()
    setobject!(vis,polyhedraMesh)
    IJuliaCell(vis)   
end

DisplayPolyhedra (generic function with 1 method)

In [13]:
function OptimizePoly(hrepPoly, objective, equals)
    hrepPoly = hrepPoly ∩ HyperPlane(objective,equals)
    model = Model()
    @variable(model,λ[1:3])
    @constraint(model, λ in hrepPoly)
    set_optimizer(model, GLPK.Optimizer)
    optimize!(model)
    optimalValues = value.(λ)
    return optimalValues
end

OptimizePoly (generic function with 1 method)

In [14]:
function main()
    objective,equals = ObjectiveIO()
    
    println("Please indicate if you would like to represent your polyhedra in either H-representation or V-representation")
    println("If you would like to use H-representation type \"H\"")
    println("If you would like to use V-representation type \"V\"")
    
    
    HorV = readline(stdin)
    
    while true
        if HorV == "H" || HorV == "V"
            break
        else
            println("Invalid character entered.")
            println("If you would like to use H-representation type \"H\"")
            println("If you would like to use V-representation type \"V\"")
            HorV = readline(stdin)
        end
    end
    
    println()
    
    if HorV == "H"
        A,b = PolyhedraIOHRep()
        halfSpacePoly = MakePolyhedra(A,b)
        vrepPoly = convertToVRep(halfSpacePoly)
    else
        vrepPoly = VertexIO()
        halfSpacePoly = ConvertToHRep(vrepPoly)
    end
    DisplayPolyhedra(halfSpacePoly)
    println("Optimal values for objective function:")
    OptimizePoly(halfSpacePoly, objective, equals)
end

main (generic function with 1 method)

In [32]:
main()

Enter the coefficients to a linear objective function
For example, if you had the objective function: 
.75x+.5y+.1z = 1
You would input: 
.75 .5 .1
stdin> .75 .5 .1


3-element Vector{Float64}:
 0.75
 0.5
 0.1

Enter what the objective function equals
For example, if you had the objective function: 
.75x+.5y+.1z = 1
You would input: 
1
stdin> 1
1.0
Please indicate if you would like to represent your polyhedra in either H-representation or V-representation
If you would like to use H-representation type "H"
If you would like to use V-representation type "V"
stdin> H

Enter the values for the coefficient matrix A of the system of linear inequalities
For example if your system looks like this: 
-x ≤ 0
-y ≤ 0
-z ≤ 0
35x+.5y+.5z ≤ .5
60x+300y+10 ≤ 15
30x+20y+10z ≤ 4
You would input: 
-1 0 0
0 -1 0
0 0 -1
35 .5 .5
 60 300 10
30 20 10
For the coefficient matrix. When you are done entering elements, press "ENTER" on the blank readline
KEEP IN MIND: The entries must be in the form of Ax ≤ b
stdin> -1 0 0
stdin> 0 -1 0
stdin> 0 0 -1
stdin> 35 .5 .5
stdin> 60 300 10
stdin> 30 20 10
stdin> 


6×3 Matrix{Float64}:
 -1.0    0.0   0.0
  0.0   -1.0   0.0
  0.0    0.0  -1.0
 35.0    0.5   0.5
 60.0  300.0  10.0
 30.0   20.0  10.0

Enter the values for vector b of the system of linear inequalities
For example if your system looks like this: 
-x ≤ 0
-y ≤ 0
-z ≤ 0
35x+.5y+.5z ≤ .5
60x+300y+10 ≤ 15
30x+20y+10z ≤ 4
You would input: 
0 0 0 .5 15 4
For b
KEEP IN MIND: The entries must be in the form of Ax ≤ b
stdin> 0 0 0 .5 15 4


...

Your polyhedra in V-representation looks like:


V-representation Polyhedra.Hull{Float64, Vector{Float64}, Int64}:
8-element iterator of Vector{Float64}:
 [0.009526343799893547, 0.03826503459286855, 0.2948908994145823]
 [0.0, 0.039285714285714285, 0.32142857142857145]
 [0.008955223880596998, 0.0, 0.37313432835820903]
 [0.01361031518624641, 0.04727793696275072, 0.0]
 [0.0, 0.05, 0.0]
 [0.014285714285714275, 0.0, 0.0]
 [0.0, 0.0, 0.4]
 [0.0, 0.0, 0.0]

Optimal values for objective function:


┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8703


3-element Vector{Float64}:
 0.00952634379989356
 0.03826503459286854
 0.29489089941458224

# 